In [7]:
import torch
import torch.nn as nn

class ConvertTo3Channels(nn.Module):
    def __init__(self):
        super(ConvertTo3Channels, self).__init__()
        self.conv = nn.Conv2d(512, 3, kernel_size=1)

    def forward(self, x):
        x = x.view(x.size(0), -1, 32, 32)
        return self.conv(x)

# Example usage
input_tensor = torch.randn(1, 512, 32, 32)  # Example input tensor

# Convert to 3 channels
converter = ConvertTo3Channels()
output_tensor = converter(input_tensor)
print("Output tensor shape:", output_tensor.shape)

Output tensor shape: torch.Size([1, 3, 32, 32])


In [2]:
output_tensor

tensor([[[[ 0.3211, -1.1504,  0.2494,  ...,  0.1789, -0.9868, -0.7163],
          [ 0.1738,  0.1289, -0.7453,  ..., -0.5446,  0.7256,  0.6766],
          [ 0.1112, -0.0700, -0.1391,  ..., -1.3333,  0.7103,  1.0882],
          ...,
          [ 0.3553, -1.4619,  0.5232,  ..., -0.1441,  0.1739, -0.1861],
          [ 0.6959,  1.1374, -1.2228,  ..., -0.0699,  0.3611,  0.1375],
          [-0.3749,  0.2975,  0.2018,  ..., -0.7373,  0.1982,  0.3421]],

         [[ 0.0651,  0.1019, -0.6330,  ..., -0.0850,  0.1727,  0.2022],
          [ 0.3702, -0.3912,  0.5050,  ..., -0.0117, -0.4342,  0.2370],
          [-0.0913, -0.2539,  0.1703,  ...,  0.4658,  0.1329, -0.0102],
          ...,
          [ 1.6260,  0.1773, -0.6413,  ...,  0.5294, -0.6304,  0.0364],
          [-0.2172,  0.0373,  0.8763,  ...,  0.2222,  0.0988,  1.0310],
          [ 0.1355, -0.2102, -0.8702,  ...,  0.2149, -0.0477,  0.7097]],

         [[ 0.2472, -0.3047,  0.4292,  ...,  1.0233, -0.7907,  0.3653],
          [-0.7131, -0.7426, -